In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## *ส่วนใหม่* *ทำข้อความเป็นตัวหนา*



In [3]:
!pip install optuna

     |████████████████████████████████| 307kB 6.6MB/s 
     |████████████████████████████████| 81kB 5.8MB/s 
     |████████████████████████████████| 174kB 8.9MB/s 
     |████████████████████████████████| 112kB 9.9MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 143kB 10.0MB/s 
     |████████████████████████████████| 81kB 6.3MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11136 sha256=8866491d498ffd8d3570316a80a7929920194811cddeffd29a8343278ad7b625
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


## **IMPORT LIBARY & DATASET**

In [4]:
import numpy as np
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report, roc_curve, auc , silhouette_samples , silhouette_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from xgboost import plot_importance
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
import joblib
from functools import *
from sklearn.feature_selection import SelectFromModel
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
from datetime import datetime

def days_between(dd):
    d1 = dd["trans_date"]
    d2 = dd["lag_trans_date"]
    return abs((d2 - d1).days)

def elbow_curve(df_input_x):
  distortions = []
  silhouettes = []
  K = range(2,15)
  for k in K:
      kmeanModel = KMeans(n_clusters=k,random_state=123)
      kmeanModel.fit(df_input_x)
      distortions.append(kmeanModel.inertia_)
      sil_avg = silhouette_score(df_input_x, kmeanModel.labels_)
      silhouettes.append(sil_avg)
  fig, (ax1, ax2) = plt.subplots(1, 2 , figsize=(12,4), tight_layout=True)
  # fig.suptitle('Horizontally Elbow x Silhouette')

  ax1.set_title("The Elbow Method showing the optimal k")
  ax1.plot(K, distortions)
  ax2.set_title("Silhouette showing the optimal k")
  ax2.plot(K, silhouettes)
  return fig.show()#(distortions,silhouettes),fig.show()

In [6]:
class Imbalace_Method :
  def __init__(self,x_data,y_data,dx_test,dy_test,fea_ls):
    self.x_data = x_data
    self.y_data = y_data
    self.dx_test = dx_test
    self.dy_test = dy_test
    self.fea_ls = fea_ls
  
  def oversam(self) :
    ros = RandomOverSampler(random_state=0)
    ros.fit(self.x_data, self.y_data)
    X_sampled, y_sampled = ros.fit_sample(self.x_data, self.y_data)
    df_sampled = pd.concat([pd.DataFrame(data=X_sampled),pd.DataFrame(data=y_sampled)], axis=1, sort=False)
    df_sampled.columns= self.fea_ls
    return df_sampled,X_sampled,y_sampled,self.dx_test,self.dy_test , "OverSampling"
  
  def downsam(self) :
    ros = RandomUnderSampler(random_state=0)
    ros.fit(self.x_data, self.y_data)
    X_sampled, y_sampled = ros.fit_sample(self.x_data, self.y_data)
    df_sampled = pd.concat([pd.DataFrame(data=X_sampled),pd.DataFrame(data=y_sampled)], axis=1, sort=False)
    df_sampled.columns= self.fea_ls
    return df_sampled,X_sampled,y_sampled,self.dx_test,self.dy_test , "UnderSampling"

  def smotesam(self) :
    sm = SMOTE(random_state=0)
    sm.fit(self.x_data, self.y_data) 
    X_sampled, y_sampled = sm.fit_sample(self.x_data, self.y_data)
    df_sampled = pd.concat([pd.DataFrame(data=X_sampled),pd.DataFrame(data=y_sampled)], axis=1, sort=False)
    df_sampled.columns= self.fea_ls
    return df_sampled,X_sampled,y_sampled,self.dx_test,self.dy_test , "Smote"

In [7]:
#class model
class MachineLearning:

  def __init__(self,X_sam_train , Y_sam_train , X_sam_test , Y_sam_test) :
    self.X_sam_train = X_sam_train
    self.Y_sam_train = Y_sam_train
    self.X_sam_test = X_sam_test
    self.Y_sam_test = Y_sam_test
    self.best_hyparam = None
    self.param_log = {
    'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
    self.param_xgb = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}  
  def build_logistic(self) :
    logreg = LogisticRegression(solver='liblinear', class_weight='balanced')
    #Part train data
    predicted_y_train = []
    expected_y_train = []
    logreg_build = logreg.fit(self.X_sam_train, self.Y_sam_train)
    predictions_train = logreg_build.predict(self.X_sam_train)
    predicted_y_train.extend(predictions_train)
    expected_y_train.extend(self.Y_sam_train)
    report_train_dic = classification_report(expected_y_train, predicted_y_train,output_dict=True)
    #Part test data
    predicted_y_test = []
    expected_y_test = []
    predictions_test = logreg_build.predict(self.X_sam_test)
    predicted_y_test.extend(predictions_test)
    expected_y_test.extend(self.Y_sam_test)
    report_test_dic = classification_report(expected_y_test, predicted_y_test,output_dict=True)
    return logreg_build,report_train_dic,report_test_dic ,"Logistic regression"

  def build_xgboost(self) :
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric="auc")
    predicted_y_train = []
    expected_y_train = []
    xgb_build = xgb_model.fit(self.X_sam_train, self.Y_sam_train, early_stopping_rounds=5, eval_set=[(self.X_sam_test.to_numpy(), self.Y_sam_test)])
    predictions_train = xgb_build.predict(self.X_sam_train)
    predicted_y_train.extend(predictions_train)
    expected_y_train.extend(self.Y_sam_train)
    report_train_dic = classification_report(expected_y_train, predicted_y_train,output_dict=True)
    #Part test data
    predicted_y_test = []
    expected_y_test = []
    predictions_test = xgb_build.predict(self.X_sam_test.to_numpy())
    predicted_y_test.extend(predictions_test)
    expected_y_test.extend(self.Y_sam_test)
    report_test_dic = classification_report(expected_y_test, predicted_y_test,output_dict=True)
    return xgb_build,report_train_dic,report_test_dic,"XGboost"

  def build_logistune(self) :
    logreg = LogisticRegression(solver='liblinear', class_weight='balanced')
    predicted_y_train = []
    expected_y_train = []
    # Create grid search object
    clf = RandomizedSearchCV(logreg, param_distributions = self.param_log , cv = 5, verbose=True, n_jobs=-1 , random_state=123)
    # Fit on data
    best_clf = clf.fit(self.X_sam_train,self.Y_sam_train)
    self.best_hyparam = best_clf.best_params_
    logreg = LogisticRegression(**self.best_hyparam)
    logreg_build = logreg.fit(self.X_sam_train,self.Y_sam_train)
    predictions_train = logreg_build.predict(self.X_sam_train)
    predicted_y_train.extend(predictions_train)
    expected_y_train.extend(self.Y_sam_train)
    report_train_dic = classification_report(expected_y_train, predicted_y_train,output_dict=True)
    #Part test data
    predicted_y_test = []
    expected_y_test = []
    predictions_test = logreg_build.predict(self.X_sam_test)
    predicted_y_test.extend(predictions_test)
    expected_y_test.extend(self.Y_sam_test)
    report_test_dic = classification_report(expected_y_test, predicted_y_test,output_dict=True)
    return logreg_build,report_train_dic,report_test_dic ,"Logistic RandomSearch"

  def build_xgbtune(self) :
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric="auc")
    predicted_y_train = []
    expected_y_train = []
    # Create grid search object
    clf = RandomizedSearchCV(xgb_model, param_distributions=self.param_xgb, random_state=42, n_iter=200, cv=3, verbose=1, n_jobs=-1, return_train_score=True)
    # Fit on data
    best_clf = clf.fit(self.X_sam_train,self.Y_sam_train)
    self.best_hyparam = best_clf.best_params_
    xgb_model = xgb.XGBClassifier(**self.best_hyparam)
    xgb_build = xgb_model.fit(self.X_sam_train,self.Y_sam_train , early_stopping_rounds=5, eval_set=[(self.X_sam_test.to_numpy(),self.Y_sam_test)])
    predictions_train = xgb_build.predict(self.X_sam_train)
    predicted_y_train.extend(predictions_train)
    expected_y_train.extend(self.Y_sam_train)
    report_train_dic = classification_report(expected_y_train, predicted_y_train,output_dict=True)
    #Part test data
    predicted_y_test = []
    expected_y_test = []
    predictions_test = xgb_build.predict(self.X_sam_test.to_numpy())
    predicted_y_test.extend(predictions_test)
    expected_y_test.extend(self.Y_sam_test)
    report_test_dic = classification_report(expected_y_test, predicted_y_test,output_dict=True)
    return xgb_build,report_train_dic,report_test_dic ,"XGboost RandomSearch"

In [8]:
# param_log = {
# 'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
# 'C' : np.logspace(-4, 4, 20),
# 'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
# }
# logreg = LogisticRegression(solver='liblinear', class_weight='balanced')
# # Create grid search object
# clf = RandomizedSearchCV(logreg, param_distributions = param_log , cv = 5, verbose=True, n_jobs=-1,random_state=123)
# # Fit on data
# best_clf = clf.fit(ls_sampling_build[0][1],ls_sampling_build[0][2])
# best_hyparam = best_clf.best_params_
# logreg = LogisticRegression(**best_hyparam)

In [9]:
# logreg_build = logreg.fit(ls_sampling_build[0][1], ls_sampling_build[0][2])
# predictions_train = logreg_build.predict(ls_sampling_build[0][1])
# predicted_y_train = []
# expected_y_train = []
# predicted_y_train.extend(predictions_train)
# expected_y_train.extend(ls_sampling_build[0][2])
# report_train_dic = classification_report(expected_y_train, predicted_y_train,output_dict=True)
# #Part test data
# predicted_y_test = []
# expected_y_test = []
# predictions_test = logreg.predict(ls_sampling_build[0][3],output_dict=True)
# predicted_y_test.extend(predictions_test)
# expected_y_test.extend(ls_sampling_build[0][4])
# report_test_dic = classification_report(expected_y_test, predicted_y_test)

In [10]:
df_response = pd.read_csv('/content/drive/MyDrive/BADS7105_CRM_ANALYTICS_AND_INTELLIGENT/HW_08_Campaign_Response_Model/Retail_Data_Response.csv')
df_transactions = pd.read_csv('/content/drive/MyDrive/BADS7105_CRM_ANALYTICS_AND_INTELLIGENT/HW_08_Campaign_Response_Model/Retail_Data_Transactions.csv', parse_dates=['trans_date'])

In [11]:
print(df_transactions["trans_date"].min() , df_transactions["trans_date"].max())

2011-05-16 00:00:00 2015-03-16 00:00:00


In [12]:
df_report_response = pd.read_excel("/content/drive/MyDrive/BADS7105_CRM_ANALYTICS_AND_INTELLIGENT/HW_08_Campaign_Response_Model/Report_Response.xlsx")

In [13]:
df_report_response

,User,Feature_Sel,N_Feature,Imbal_Method,ML,Tune_Flag,Train_Acc,Train_Rec_Negative,Train_Pre_Negative,Train_Rec_Positive,Train_Pre_Positive,Test_Acc,Test_Rec_Negative,Test_Pre_Negative,Test_Rec_Positive,Test_Pre_Positive
0,AJ,N,3,UnderSampling,Logistic regression,N,0.670163,0.620047,0.689119,0.720280,0.654661,0.617135,0.600108,0.955211,0.761468,0.183425
1,AJ,N,3,OverSampling,Logistic regression,N,0.658578,0.623149,0.670672,0.694008,0.648085,0.642304,0.633117,0.950447,0.720183,0.188024
2,AJ,N,3,Smote,Logistic regression,N,0.666097,0.619275,0.683258,0.712919,0.651875,0.635528,0.626623,0.948403,0.711009,0.183432
3,AJ,N,3,UnderSampling,XGboost,N,0.715618,0.608392,0.774481,0.822844,0.677543,0.562924,0.534632,0.958293,0.802752,0.169082
4,AJ,N,3,OverSampling,XGboost,N,0.682274,0.536569,0.757235,0.827979,0.641143,0.560019,0.529221,0.961652,0.821101,0.170639
5,AJ,N,3,Smote,XGboost,N,0.727956,0.594668,0.810811,0.861244,0.679978,0.617135,0.602814,0.951324,0.738532,0.179888
6,AJ,N,3,UnderSampling,XGboost,Y,0.741627,0.626566,0.813850,0.856687,0.696425,0.646176,0.634740,0.954434,0.743119,0.193548


## **CLASS PREP DATA**

In [14]:
class PrebData :

  def __init__(self,df_init,df_res,label_dset="") :
    self.campaign_date = dt.datetime(2015,3,17)
    self.label_dset = label_dset
    self.df_init = df_init
    self.df_init['recent']= self.campaign_date - self.df_init['trans_date']
    self.df_init['recent'].astype('timedelta64[D]')
    self.df_init['recent']=self.df_init['recent'] / np.timedelta64(1, 'D')
    self.df_init["Year"] = pd.DatetimeIndex(self.df_init['trans_date']).year
    self.df_init["Month"] = pd.DatetimeIndex(self.df_init['trans_date']).month
    self.df_init["Day"] = pd.DatetimeIndex(self.df_init['trans_date']).day
    self.df_init['weekday'] = self.df_init[['trans_date']].apply(lambda x: dt.datetime.strftime(x['trans_date'], '%A'), axis=1)
    self.df_rfm = None
    self.df_agg_day = None
    self.pivot_df_agg_weekday = None
    self.df_time2event = None
    self.df_agg_weekday = None
    self.pivot_df_agg_weekday = None
    self.df_time2event = None
    self.df_tk_size = None
    self.df_tk_size_stat = None
    self.df_tk_size_stat_agg = None
    self.pivot_table_amt_sum = None
    self.pivot_table_amt_avg = None
    self.pivot_table_txn_cnt = None
    self.df_merged = None
    self.df_res = df_res

  def build_rfm(self):
    self.df_rfm = self.df_init.groupby('customer_id').agg({'recent': lambda x:x.min(),                      # Recency
                                                        'customer_id': lambda x: len(x),                 # Frequency
                                                        'tran_amount': lambda x: x.sum()}) # Monetary Value
    self.df_rfm.rename(columns={'recent': 'recency', 
                            'customer_id': 'frequency', 
                            'tran_amount': 'monetary_value'}, inplace=True)
  def build_agg_day(self) :
    self.df_agg_day = self.df_init[["customer_id","trans_date","tran_amount"]].groupby(["customer_id","trans_date"]).mean()
    self.df_agg_day = self.df_agg_day.reset_index()[["customer_id","trans_date"]].sort_values(by=["customer_id","trans_date"])
    self.df_agg_day["shift_trans_date"] = self.df_agg_day['trans_date'].shift(1)
    self.df_agg_day["CHECK_FIRST"] = self.df_agg_day.duplicated("customer_id")
    self.df_agg_day["lag_trans_date"] = np.where(self.df_agg_day["CHECK_FIRST"]==False , np.datetime64('NaT') , self.df_agg_day["shift_trans_date"] )
    self.df_agg_day = self.df_agg_day[~self.df_agg_day["shift_trans_date"].isnull()]
    self.df_agg_day = self.df_agg_day[["customer_id","trans_date","lag_trans_date"]]
    self.df_agg_day["date_diff"] =self.df_agg_day.apply(days_between,axis=1)

  def build_agg_weekday(self):
    self.df_agg_weekday = self.df_init[["customer_id","trans_date","tran_amount"]].groupby(["customer_id","trans_date"]).mean()
    self.df_agg_weekday = self.df_agg_weekday.reset_index()[["customer_id","trans_date"]].sort_values(by=["customer_id","trans_date"])
    self.df_agg_weekday['weekday'] = self.df_agg_weekday[['trans_date']].apply(lambda x: dt.datetime.strftime(x['trans_date'], '%A'), axis=1)
    self.pivot_df_agg_weekday = pd.pivot_table (self.df_agg_weekday[["customer_id","trans_date","weekday"]], values='trans_date', index=['customer_id'],
                      columns=['weekday'], aggfunc="count").reset_index().fillna(0)
    self.pivot_df_agg_weekday["normal_day"] = self.pivot_df_agg_weekday["Monday"] + self.pivot_df_agg_weekday["Tuesday"] + self.pivot_df_agg_weekday["Wednesday"] + self.pivot_df_agg_weekday["Thursday"]
    self.pivot_df_agg_weekday["weekend_day"] = self.pivot_df_agg_weekday["Friday"] + self.pivot_df_agg_weekday["Saturday"] + self.pivot_df_agg_weekday["Sunday"]
    self.pivot_df_agg_weekday["normal_prop"] = self.pivot_df_agg_weekday["normal_day"]/(self.pivot_df_agg_weekday["normal_day"]+self.pivot_df_agg_weekday["weekend_day"])
    self.pivot_df_agg_weekday["weekend_prop"] = self.pivot_df_agg_weekday["weekend_day"]/(self.pivot_df_agg_weekday["normal_day"]+self.pivot_df_agg_weekday["weekend_day"])
    self.pivot_df_agg_weekday.columns = [i+"_cntday" if i != "customer_id" else i for i in self.pivot_df_agg_weekday.columns]

  def build_time2event(self):
    self.df_agg_day = self.df_agg_day.merge(self.df_agg_day.groupby("customer_id").quantile(.25).reset_index()[["customer_id","date_diff"]].rename({"date_diff":"pc25_datediff"},axis=1),how="left",on="customer_id")
    self.df_agg_day = self.df_agg_day.merge(self.df_agg_day.groupby("customer_id").quantile(.75).reset_index()[["customer_id","date_diff"]].rename({"date_diff":"pc75_datediff"},axis=1),how="left",on="customer_id")
    self.df_time2event = self.df_agg_day[(self.df_agg_day["date_diff"] >= self.df_agg_day["pc25_datediff"]) & (self.df_agg_day["date_diff"] <= self.df_agg_day["pc75_datediff"])].groupby("customer_id").mean().reset_index().rename({"date_diff":"time_to_event"},axis=1)[["customer_id","time_to_event"]]

  def build_tk_size(self):
    self.df_tk_size = self.df_init.merge(self.df_init.groupby("customer_id").quantile(.25).reset_index()[["customer_id","tran_amount"]].rename({"tran_amount":"pc25_amount"},axis=1),how="left",on="customer_id")
    self.df_tk_size = self.df_tk_size.merge(self.df_init.groupby("customer_id").quantile(.75).reset_index()[["customer_id","tran_amount"]].rename({"tran_amount":"pc75_amount"},axis=1),how="left",on="customer_id")
    self.df_tk_size_stat = self.df_tk_size[(self.df_tk_size["tran_amount"] >= self.df_tk_size["pc25_amount"]) & (self.df_tk_size["tran_amount"] <= self.df_tk_size["pc75_amount"])]
    self.df_tk_size_stat_agg = self.df_tk_size_stat[["customer_id","tran_amount"]].groupby("customer_id").mean().rename({"tran_amount":"avg_amt"},axis=1).reset_index()
    self.df_tk_size_stat_agg = self.df_tk_size_stat_agg.merge(self.df_tk_size_stat[["customer_id","tran_amount"]].groupby("customer_id").std().rename({"tran_amount":"std_amt"},axis=1).reset_index(),how="left",on="customer_id")
    # print(5)
    # self.df_tk_size_stat_agg = self.df_tk_size_stat_agg.merge(self.df_tk_size_stat_agg[["customer_id","tran_amount"]].groupby("customer_id").std().rename({"tran_amount":"std_amt"},axis=1).reset_index(),how="left",on="customer_id")

  def build_week_day_amtsum(self):
    self.pivot_table_amt_sum = pd.pivot_table(self.df_init[["customer_id","tran_amount","weekday"]], values='tran_amount', index=['customer_id'],
                      columns=['weekday'], aggfunc=np.sum).reset_index().fillna(0)
    self.pivot_table_amt_sum["normal_day"] = self.pivot_table_amt_sum["Monday"] + self.pivot_table_amt_sum["Tuesday"] + self.pivot_table_amt_sum["Wednesday"] + self.pivot_table_amt_sum["Thursday"]
    self.pivot_table_amt_sum["weekend_day"] = self.pivot_table_amt_sum["Friday"] + self.pivot_table_amt_sum["Saturday"] + self.pivot_table_amt_sum["Sunday"]
    self.pivot_table_amt_sum["normal_prop"] = self.pivot_table_amt_sum["normal_day"]/(self.pivot_table_amt_sum["normal_day"]+self.pivot_table_amt_sum["weekend_day"])
    self.pivot_table_amt_sum["weekend_prop"] = self.pivot_table_amt_sum["weekend_day"]/(self.pivot_table_amt_sum["normal_day"]+self.pivot_table_amt_sum["weekend_day"])
    self.pivot_table_amt_sum.columns = [i+"_sum" if i != "customer_id" else i for i in self.pivot_table_amt_sum.columns]

  def build_week_day_amtavg(self):
    self.pivot_table_amt_avg = pd.pivot_table(self.df_init[["customer_id","tran_amount","weekday"]], values='tran_amount', index=['customer_id'],
                      columns=['weekday'], aggfunc=np.mean).reset_index().fillna(0)
    self.pivot_table_amt_avg["normal_day"] = (self.pivot_table_amt_avg["Monday"] + self.pivot_table_amt_avg["Tuesday"] + self.pivot_table_amt_avg["Wednesday"] + self.pivot_table_amt_avg["Thursday"])/4
    self.pivot_table_amt_avg["weekend_day"] = (self.pivot_table_amt_avg["Friday"] + self.pivot_table_amt_avg["Saturday"] + self.pivot_table_amt_avg["Sunday"])/3
    self.pivot_table_amt_avg["normal_prop"] = np.where(self.pivot_table_amt_avg["normal_day"]/self.pivot_table_amt_avg["weekend_day"] >= 10 , 10 , self.pivot_table_amt_avg["normal_day"]/self.pivot_table_amt_avg["weekend_day"])
    self.pivot_table_amt_avg["weekend_prop"] = np.where(self.pivot_table_amt_avg["weekend_day"]/self.pivot_table_amt_avg["normal_day"] >= 10 , 10 , self.pivot_table_amt_avg["weekend_day"]/self.pivot_table_amt_avg["normal_day"])
    self.pivot_table_amt_avg.columns = [i+"_avg" if i != "customer_id" else i for i in self.pivot_table_amt_avg.columns]

  def build_week_day_cnt(self):
    self.pivot_table_txn_cnt = pd.pivot_table(self.df_init[["customer_id","tran_amount","weekday"]], values='tran_amount', index=['customer_id'],
                        columns=['weekday'], aggfunc="count").reset_index().fillna(0)
    self.pivot_table_txn_cnt["normal_day"] = self.pivot_table_txn_cnt["Monday"] + self.pivot_table_txn_cnt["Tuesday"] + self.pivot_table_txn_cnt["Wednesday"] + self.pivot_table_txn_cnt["Thursday"]
    self.pivot_table_txn_cnt["weekend_day"] = self.pivot_table_txn_cnt["Friday"] + self.pivot_table_txn_cnt["Saturday"] + self.pivot_table_txn_cnt["Sunday"]
    self.pivot_table_txn_cnt["normal_prop"] = self.pivot_table_txn_cnt["normal_day"]/(self.pivot_table_txn_cnt["normal_day"]+self.pivot_table_txn_cnt["weekend_day"])
    self.pivot_table_txn_cnt["weekend_prop"] = self.pivot_table_txn_cnt["weekend_day"]/(self.pivot_table_txn_cnt["normal_day"]+self.pivot_table_txn_cnt["weekend_day"])
    self.pivot_table_txn_cnt.columns = [i+"_cnt" if i != "customer_id" else i for i in self.pivot_table_txn_cnt.columns]

  def stroke_data(self,var_sel = "n") :
    def lasso_selection(df) :
      X_train, X_test, y_train, y_test = train_test_split(
        df.drop(labels=['customer_id',"response"], axis=1),
        df['response'],
        test_size=0.3,random_state=123)
      scaler = StandardScaler()
      scaler.fit(X_train.fillna(0))
      sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1',solver="liblinear"))
      sel_.fit(scaler.transform(X_train.fillna(0)), y_train)
      selected_feat = X_train.columns[(sel_.get_support())]
      removed_feats = list(X_train.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()])
      return removed_feats

    def rfr_selection(df) :
      X_train, X_test, y_train, y_test = train_test_split(
        df.drop(labels=['customer_id',"response"], axis=1),
        df['response'],
        test_size=0.3,random_state=123)
      scaler = StandardScaler()
      scaler.fit(X_train.fillna(0))
      sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
      sel.fit(X_train, y_train)
      selected_feat= X_train.columns[(sel.get_support())]
      removed_feats = [ i for i in X_train.columns if i not in list(selected_feat)]
      return removed_feats
    ls_all_df = [
                 self.df_rfm ,  self.pivot_df_agg_weekday ,  self.df_time2event  ,  self.df_tk_size_stat_agg ,  self.pivot_table_amt_sum ,  self.pivot_table_amt_avg ,  self.pivot_table_txn_cnt 
                 ]
    self.df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['customer_id'],
                                            how='left'), ls_all_df)
    self.df_merged = self.df_merged.merge(self.df_res,how="inner", on="customer_id")
    if var_sel.lower() == "lasso" :
      self.df_merged =  self.df_merged.drop(lasso_selection(self.df_merged),axis=1)
    elif var_sel.lower() == "rfr" :
      self.df_merged =  self.df_merged.drop(rfr_selection(self.df_merged),axis=1)

    return self.df_merged , var_sel.lower() , self.label_dset
    

In [15]:
# df_transactions_3y = campaign_date = dt.datetime(2015,3,17)
df_transactions_2y = df_transactions[df_transactions["trans_date"] >= dt.datetime(2013,1,1)]
df_transactions_3y = df_transactions[df_transactions["trans_date"] >= dt.datetime(2012,1,1)]

## **DATA PREPARATION**

In [16]:
df_master_all = PrebData(df_transactions , df_response)
df_master_2yr = PrebData(df_transactions_2y , df_response)
df_master_3yr = PrebData(df_transactions_3y , df_response)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [17]:
%%time
ls_data_set = [df_transactions,df_transactions_2y,df_transactions_3y]
ls_data_labels = ["df_transactions","df_transactions_2y","df_transactions_3y"]
ls_fea_sel = ["n","lasso","rfr"]
ls_data_agg = []
rnd_label = 0
for dset in ls_data_set :
  df_master_all = PrebData(df_transactions , df_response , ls_data_labels[rnd_label])
  df_master_all.build_rfm()
  df_master_all.build_agg_day()
  df_master_all.build_agg_weekday()
  df_master_all.build_time2event()
  df_master_all.build_tk_size()
  df_master_all.build_week_day_amtsum()
  df_master_all.build_week_day_amtavg()
  df_master_all.build_week_day_cnt()
  for fts in ls_fea_sel :
    ls_data_agg.append(df_master_all.stroke_data(fts))
    if fts == "n" :
      ls_data_agg.append((df_master_all.stroke_data("n")[0][["customer_id","recency","frequency","monetary_value","response"]],df_master_all.stroke_data("n")[1],df_master_all.stroke_data("n")[2]+"_rfm"))
      ls_data_agg.append((df_master_all.stroke_data("n")[0][["customer_id","recency","frequency","monetary_value","time_to_event","response"]],df_master_all.stroke_data("n")[1],df_master_all.stroke_data("n")[2]+"_rfm2ev"))
  rnd_label += 1

CPU times: user 45.3 s, sys: 435 ms, total: 45.7 s
Wall time: 45.7 s


In [18]:
import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

--- 3.1948089599609375e-05 seconds ---


In [19]:
(time.time() - start_time)

0.014265060424804688

## **TRAIN**

In [20]:
## inputdata / Feature Selection Input
%%time
rnd=0
ls_sampling_build = []
ls_model_build = []
ls_combination_build = []
ls_time2build = []
for dsetsa in ls_data_agg :
  X = dsetsa[0].drop(["customer_id","response"],axis=1)
  y = dsetsa[0]['response']
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
  ls_sel_new = list(X.columns) + ["response"]
  # print(len(ls_sel_new),ls_sel_new)
  for i in range(3) :
    imbal_master = Imbalace_Method(X_train,y_train,X_test,y_test,ls_sel_new)
    if i == 0 :
      smpling_build = imbal_master.oversam()
      ls_sampling_build.append(smpling_build)
    elif i == 1 :
      smpling_build = imbal_master.downsam()
      ls_sampling_build.append(smpling_build)
    elif i == 2 :
      smpling_build = imbal_master.smotesam()
      ls_sampling_build.append(smpling_build)
    ls_combination_build.append([dsetsa[2],dsetsa[1], smpling_build[5]])
rnd_ta = 0
for j in ls_sampling_build :
  start_time = time.time()
  ml_master = MachineLearning(j[1],j[2],j[3],j[4])
  for k in range(4) :
    if k == 0 :
      ml_build = ml_master.build_logistic() # df , x_sam_train , y_sam_train , label_imbal_method
      ls_model_build.append((j,ml_build,ls_combination_build[rnd_ta][0],ls_combination_build[rnd_ta][1]))
      ls_time2build.append((time.time() - start_time))
    elif k == 1 :
      ml_build = ml_master.build_xgboost() # df , x_sam_train , y_sam_train , label_imbal_method
      ls_model_build.append((j,ml_build,ls_combination_build[rnd_ta][0],ls_combination_build[rnd_ta][1]))
      ls_time2build.append((time.time() - start_time))
    elif k == 2 :
      ml_build = ml_master.build_logistune() # df , x_sam_train , y_sam_train , label_imbal_method
      ls_model_build.append((j,ml_build,ls_combination_build[rnd_ta][0],ls_combination_build[rnd_ta][1]))
      ls_time2build.append((time.time() - start_time))
    elif k == 3 :
      ml_build = ml_master.build_xgbtune() # df , x_sam_train , y_sam_train , label_imbal_method
      ls_model_build.append((j,ml_build,ls_combination_build[rnd_ta][0],ls_combination_build[rnd_ta][1]))
      ls_time2build.append((time.time() - start_time))
    rnd +=1
  rnd_ta += 1
  # break
    # print(rnd)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.2

[0]	validation_0-auc:0.713526
Will train until validation_0-auc hasn't improved in 5 rounds.
[1]	validation_0-auc:0.710211
[2]	validation_0-auc:0.723958
[3]	validation_0-auc:0.725204
[4]	validation_0-auc:0.731052
[5]	validation_0-auc:0.73135
[6]	validation_0-auc:0.734444
[7]	validation_0-auc:0.739324
[8]	validation_0-auc:0.736131
[9]	validation_0-auc:0.737727
[10]	validation_0-auc:0.733883
[11]	validation_0-auc:0.732654
[12]	validation_0-auc:0.734934
Stopping. Best iteration:
[7]	validation_0-auc:0.739324

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   38.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   38.9s finished


Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed: 12.2min finished


...

[4]	validation_0-error:0.469506
[5]	validation_0-error:0.471926
[6]	validation_0-error:0.468538
[7]	validation_0-error:0.458858
[8]	validation_0-error:0.461278
[9]	validation_0-error:0.45547
[10]	validation_0-error:0.461762
[11]	validation_0-error:0.458374
[12]	validation_0-error:0.449177
[13]	validation_0-error:0.450145
[14]	validation_0-error:0.450145
[15]	validation_0-error:0.450629
[16]	validation_0-error:0.449177
[17]	validation_0-error:0.446273
[18]	validation_0-error:0.439497
[19]	validation_0-error:0.439013
[20]	validation_0-error:0.441433
[21]	validation_0-error:0.440949
[22]	validation_0-error:0.440465
[23]	validation_0-error:0.439497
[24]	validation_0-error:0.438045
[25]	validation_0-error:0.43756
[26]	validation_0-error:0.434172
[27]	validation_0-error:0.4303
[28]	validation_0-error:0.4303
[29]	validation_0-error:0.42788
[30]	validation_0-error:0.429816
[31]	validation_0-error:0.431268
[32]	validation_0-error:0.43272
[33]	validation_0-error:0.433688
[34]	validation_0-error:

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   19.8s finished
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning:

Setting penalty='none' will ignore the C and l1_ratio parameters

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning:

Setting penalty='none' will ignore the C and l1_ratio parameters



Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   55.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed: 11.0min finished


[0]	validation_0-error:0.406099
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.336883
[2]	validation_0-error:0.333979
[3]	validation_0-error:0.328654
[4]	validation_0-error:0.31365
[5]	validation_0-error:0.298645
[6]	validation_0-error:0.272507
[7]	validation_0-error:0.270571
[8]	validation_0-error:0.260891
[9]	validation_0-error:0.254598
[10]	validation_0-error:0.258954
[11]	validation_0-error:0.25847
[12]	validation_0-error:0.255566
[13]	validation_0-error:0.253146
[14]	validation_0-error:0.24879
[15]	validation_0-error:0.244918
[16]	validation_0-error:0.242498
[17]	validation_0-error:0.238141
[18]	validation_0-error:0.236689
[19]	validation_0-error:0.228461
[20]	validation_0-error:0.231365
[21]	validation_0-error:0.232333
[22]	validation_0-error:0.229429
[23]	validation_0-error:0.227977
[24]	validation_0-error:0.219264
[25]	validation_0-error:0.219264
[26]	validation_0-error:0.21878
[27]	validation_0-error:0.213456
[28]	validation_0-error:0

In [25]:
ttx = 0
for jj in ls_model_build :
  print(jj[2] , jj[3] , jj[0][5] , jj[1][3] , jj[1][2]["accuracy"] ,ls_time2build[ttx])
  ttx+=1

df_transactions n OverSampling Logistic regression 0.6321393998063891 1.166560411453247
df_transactions n OverSampling XGboost 0.6229428848015489 1.6728429794311523
df_transactions n OverSampling Logistic RandomSearch 0.6379477250726041 42.45768141746521
df_transactions n OverSampling XGboost RandomSearch 0.8673765730880929 779.94180560112
df_transactions n UnderSampling Logistic regression 0.6181026137463698 0.08570504188537598
df_transactions n UnderSampling XGboost 0.5726040658276863 0.26141858100891113
df_transactions n UnderSampling Logistic RandomSearch 0.6263310745401742 4.542601823806763
df_transactions n UnderSampling XGboost RandomSearch 0.6069699903194579 103.97650647163391
df_transactions n Smote Logistic regression 0.6340755082284608 0.9580609798431396
df_transactions n Smote XGboost 0.6655372700871249 1.5558443069458008
df_transactions n Smote Logistic RandomSearch 0.6345595353339787 36.44343185424805
df_transactions n Smote XGboost RandomSearch 0.6752178121974831 992.044

In [22]:
ls_user = []
ls_featuresel = []
ls_nfeature = []
ls_imbalmeth = []
ls_ml = []
ls_tune = []
ls_train_acc = []
ls_train_rec_0 = []
ls_train_pre_0 = []
ls_train_rec_1 = []
ls_train_pre_1 = []
ls_test_acc = []
ls_test_rec_0 = []
ls_test_pre_0 = []
ls_test_rec_1 = []
ls_test_pre_1 = []
for bb in ls_model_build :
  ls_user.append(bb[2])
  ls_featuresel.append(bb[3])
  ls_nfeature.append(bb[0][1].shape[1])
  ls_imbalmeth.append(bb[0][5])
  ls_ml.append(bb[1][3])
  ls_tune.append("N")
  ls_train_acc.append(bb[1][1]["accuracy"])
  ls_train_rec_0.append(bb[1][1]["0"]["recall"])
  ls_train_pre_0.append(bb[1][1]["0"]["precision"])
  ls_train_rec_1.append(bb[1][1]["1"]["recall"])
  ls_train_pre_1.append(bb[1][1]["1"]["precision"])
  ls_test_acc.append(bb[1][2]["accuracy"])
  ls_test_rec_0.append(bb[1][2]["0"]["recall"])
  ls_test_pre_0.append(bb[1][2]["0"]["precision"])
  ls_test_rec_1.append(bb[1][2]["1"]["recall"])
  ls_test_pre_1.append(bb[1][2]["1"]["precision"])

In [26]:
df_base_mytest = pd.DataFrame({
    "User" : ls_user , 
    "Feature_Sel" : ls_featuresel , 
    "N_Feature" : ls_nfeature ,
    "Imbal_Method" : ls_imbalmeth ,
    "ML" : ls_ml ,
    "Tune_Flag" : ls_tune ,
    "Train_Acc" : ls_train_acc ,
    "Train_Rec_Negative" : ls_train_rec_0 ,
    "Train_Pre_Negative" : ls_train_pre_0 ,
    "Train_Rec_Positive" : ls_train_rec_1 ,
    "Train_Pre_Positive" : ls_train_pre_1 , 
    "Test_Acc" : ls_test_acc ,
    "Test_Rec_Negative" : ls_test_rec_0 ,
    "Test_Pre_Negative" : ls_test_pre_0 ,
    "Test_Rec_Positive" : ls_test_rec_1 ,
    "Test_Pre_Positive" : ls_test_pre_1 ,
    "Time2Build" : ls_time2build
})

In [27]:
df_base_mytest = df_base_mytest.sort_values(by=["Test_Acc"],ascending=False)

In [28]:
df_base_mytest.shape

(180, 17)

In [1]:
ls_model_build[0][1][3]

NameError: ignored

In [29]:
df_report_response

,User,Feature_Sel,N_Feature,Imbal_Method,ML,Tune_Flag,Train_Acc,Train_Rec_Negative,Train_Pre_Negative,Train_Rec_Positive,Train_Pre_Positive,Test_Acc,Test_Rec_Negative,Test_Pre_Negative,Test_Rec_Positive,Test_Pre_Positive
0,AJ,N,3,UnderSampling,Logistic regression,N,0.670163,0.620047,0.689119,0.720280,0.654661,0.617135,0.600108,0.955211,0.761468,0.183425
1,AJ,N,3,OverSampling,Logistic regression,N,0.658578,0.623149,0.670672,0.694008,0.648085,0.642304,0.633117,0.950447,0.720183,0.188024
2,AJ,N,3,Smote,Logistic regression,N,0.666097,0.619275,0.683258,0.712919,0.651875,0.635528,0.626623,0.948403,0.711009,0.183432
3,AJ,N,3,UnderSampling,XGboost,N,0.715618,0.608392,0.774481,0.822844,0.677543,0.562924,0.534632,0.958293,0.802752,0.169082
4,AJ,N,3,OverSampling,XGboost,N,0.682274,0.536569,0.757235,0.827979,0.641143,0.560019,0.529221,0.961652,0.821101,0.170639
5,AJ,N,3,Smote,XGboost,N,0.727956,0.594668,0.810811,0.861244,0.679978,0.617135,0.602814,0.951324,0.738532,0.179888
6,AJ,N,3,UnderSampling,XGboost,Y,0.741627,0.626566,0.813850,0.856687,0.696425,0.646176,0.634740,0.954434,0.743119,0.193548


In [35]:
def display_metrics(df):
    return df.style.bar(subset=['Train_Acc'], color='lightskyblue')\
                    .bar(subset=['Test_Acc'], color='lightgreen')\
                    .bar(subset=['Time2Build'], color='orange')\
                    .format({'Train_Acc':'{:.2%}','Test_Acc':'{:.2%}','Time2Build':'{:}'})
df_filter_got_result = df_base_mytest[["User","Feature_Sel","N_Feature","Imbal_Method","ML","Train_Acc","Test_Acc","Time2Build"]]
display_metrics(df_filter_got_result[df_filter_got_result["Test_Acc"] >= np.max(df_report_response["Test_Acc"])])

,User,Feature_Sel,N_Feature,Imbal_Method,ML,Train_Acc,Test_Acc,Time2Build
107,df_transactions_2y,lasso,29,Smote,XGboost RandomSearch,95.73%,88.96%,629.8089141845703
47,df_transactions,lasso,29,Smote,XGboost RandomSearch,95.73%,88.96%,642.0807332992554
167,df_transactions_3y,lasso,29,Smote,XGboost RandomSearch,95.73%,88.96%,650.1938438415527
3,df_transactions,n,50,OverSampling,XGboost RandomSearch,99.92%,86.74%,779.94180560112
63,df_transactions_2y,n,50,OverSampling,XGboost RandomSearch,99.92%,86.74%,752.6027274131775
123,df_transactions_3y,n,50,OverSampling,XGboost RandomSearch,99.92%,86.74%,758.3046164512634
111,df_transactions_2y,rfr,27,OverSampling,XGboost RandomSearch,99.32%,85.04%,523.2242650985718
99,df_transactions_2y,lasso,29,OverSampling,XGboost RandomSearch,99.48%,84.95%,515.8789756298065
159,df_transactions_3y,lasso,29,OverSampling,XGboost RandomSearch,99.48%,84.95%,513.8290205001831
39,df_transactions,lasso,29,OverSampling,XGboost RandomSearch,99.48%,84.95%,527.9752542972565


(125000, 8)

In [121]:
ls_model_build[0][0][3]

,recency,frequency,monetary_value,Friday_cntday,Monday_cntday,Saturday_cntday,Sunday_cntday,Thursday_cntday,Tuesday_cntday,Wednesday_cntday,normal_day_cntday,weekend_day_cntday,normal_prop_cntday,weekend_prop_cntday,time_to_event,avg_amt,std_amt,Friday_sum,Monday_sum,Saturday_sum,Sunday_sum,Thursday_sum,Tuesday_sum,Wednesday_sum,normal_day_sum,weekend_day_sum,normal_prop_sum,weekend_prop_sum,Friday_avg,Monday_avg,Saturday_avg,Sunday_avg,Thursday_avg,Tuesday_avg,Wednesday_avg,normal_day_avg,weekend_day_avg,normal_prop_avg,weekend_prop_avg,Friday_cnt,Monday_cnt,Saturday_cnt,Sunday_cnt,Thursday_cnt,Tuesday_cnt,Wednesday_cnt,normal_day_cnt,weekend_day_cnt,normal_prop_cnt,weekend_prop_cnt
5546,28.0,18,788,1.0,1.0,4.0,2.0,6.0,3.0,1.0,11.0,7.0,0.611111,0.388889,65.333333,43.000000,8.992060,65.0,29.0,150.0,88.0,263.0,120.0,73.0,485.0,303.0,0.615482,0.384518,65.000000,29.000000,37.500000,44.000000,43.833333,40.000000,73.000000,46.458333,48.833333,0.951365,1.051121,1.0,1.0,4.0,2.0,6.0,3.0,1.0,11.0,7.0,0.611111,0.388889
3027,158.0,31,2099,3.0,5.0,6.0,6.0,7.0,4.0,0.0,16.0,15.0,0.516129,0.483871,22.928571,66.466667,9.927355,238.0,305.0,334.0,427.0,425.0,370.0,0.0,1100.0,999.0,0.524059,0.475941,79.333333,61.000000,55.666667,71.166667,60.714286,92.500000,0.000000,53.553571,68.722222,0.779276,1.283243,3.0,5.0,6.0,6.0,7.0,4.0,0.0,16.0,15.0,0.516129,0.483871
6234,90.0,13,694,5.0,1.0,1.0,2.0,0.0,1.0,3.0,5.0,8.0,0.384615,0.615385,96.166667,56.142857,15.071259,313.0,79.0,18.0,81.0,0.0,41.0,162.0,282.0,412.0,0.406340,0.593660,62.600000,79.000000,18.000000,40.500000,0.000000,41.000000,54.000000,43.500000,40.366667,1.077622,0.927969,5.0,1.0,1.0,2.0,0.0,1.0,3.0,5.0,8.0,0.384615,0.615385
4308,19.0,18,1156,3.0,2.0,3.0,1.0,4.0,3.0,2.0,11.0,7.0,0.611111,0.388889,65.000000,59.250000,7.796519,161.0,158.0,241.0,77.0,191.0,220.0,108.0,677.0,479.0,0.585640,0.414360,53.666667,79.000000,80.333333,77.000000,47.750000,73.333333,54.000000,63.520833,70.333333,0.903140,1.107248,3.0,2.0,3.0,1.0,4.0,3.0,2.0,11.0,7.0,0.611111,0.388889
3035,25.0,25,1739,3.0,8.0,2.0,6.0,4.0,0.0,2.0,14.0,11.0,0.560000,0.440000,33.833333,69.692308,8.330512,199.0,494.0,144.0,408.0,320.0,0.0,174.0,988.0,751.0,0.568143,0.431857,66.333333,61.750000,72.000000,68.000000,80.000000,0.000000,87.000000,57.187500,68.777778,0.831482,1.202672,3.0,8.0,2.0,6.0,4.0,0.0,2.0,14.0,11.0,0.560000,0.440000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,97.0,10,749,0.0,1.0,1.0,0.0,2.0,1.0,5.0,9.0,1.0,0.900000,0.100000,37.000000,78.000000,17.068489,0.0,71.0,41.0,0.0,150.0,99.0,388.0,708.0,41.0,0.945260,0.054740,0.000000,71.000000,41.000000,0.000000,75.000000,99.000000,77.600000,80.650000,13.666667,5.901220,0.169456,0.0,1.0,1.0,0.0,2.0,1.0,5.0,9.0,1.0,0.900000,0.100000
1303,42.0,25,1821,3.0,3.0,3.0,3.0,4.0,6.0,3.0,16.0,9.0,0.640000,0.360000,32.666667,71.538462,11.027355,247.0,209.0,204.0,233.0,304.0,473.0,151.0,1137.0,684.0,0.624382,0.375618,82.333333,69.666667,68.000000,77.666667,76.000000,78.833333,50.333333,68.708333,76.000000,0.904057,1.106125,3.0,3.0,3.0,3.0,4.0,6.0,3.0,16.0,9.0,0.640000,0.360000
474,74.0,20,1441,5.0,2.0,2.0,5.0,4.0,1.0,1.0,8.0,12.0,0.400000,0.600000,37.222222,69.900000,10.375720,360.0,191.0,129.0,353.0,270.0,97.0,41.0,599.0,842.0,0.415684,0.584316,72.000000,95.500000,64.500000,70.600000,67.500000,97.000000,41.000000,75.250000,69.033333,1.090053,0.917386,5.0,2.0,2.0,5.0,4.0,1.0,1.0,8.0,12.0,0.400000,0.600000
1789,191.0,19,1259,4.0,5.0,2.0,2.0,0.0,5.0,1.0,11.0,8.0,0.578947,0.421053,58.750000,63.777778,9.510228,271.0,306.0,127.0,151.0,0.0,328.0,76.0,710.0,549.0,0.563940,0.436060,67.750000,61.200000,63.500000,75.500000,0.000000,65.600000,76.000000,50.700000,68.916667,0.735671,1.359303,4.0,5.0,2.0,2.0,0.0,5.0,1.0,11.0,8.0,0.578947,0.421053


In [122]:
best_clf.best_params_

NameError: ignored

In [ ]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
xgb_cls = xgb.XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb_cls, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train, y_train), verbose=3, random_state=1001 )

# Here we go
# start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train, y_train)
# timer(start_time) # timing ends here for "start_time" variable
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)

In [ ]:
print('logistic regression model - oversampled')
logreg = LogisticRegression(C=best_clf.best_params_["C"] , solver=best_clf.best_params_["solver"], class_weight='balanced',penalty="l1")
predicted_y = []
expected_y = []

logreg_model_over = logreg.fit(ls_sampling_build[0][1],ls_sampling_build[0][2])
predictions = logreg_model_over.predict(ls_sampling_build[0][1])
predicted_y.extend(predictions)
expected_y.extend(ls_sampling_build[0][2])
report_train = classification_report(expected_y, predicted_y)
print('training set')
print(report_train) 

predicted_y = []
expected_y = []
predictions = logreg_model_over.predict(X_test)
predicted_y.extend(predictions)
expected_y.extend(y_test)
report_test = classification_report(expected_y, predicted_y)
print('test set')
print(report_test)

In [ ]:
print('logistic regression model - oversampled')
logreg = LogisticRegression(solver='liblinear', class_weight='balanced')
predicted_y = []
expected_y = []

logreg_model_over = logreg.fit(ls_sampling_build[0][1],ls_sampling_build[0][2])
predictions = logreg_model_over.predict(ls_sampling_build[0][1])
predicted_y.extend(predictions)
expected_y.extend(ls_sampling_build[0][2])
report_train = classification_report(expected_y, predicted_y)
print('training set')
print(report_train) 

predicted_y = []
expected_y = []
predictions = logreg_model_over.predict(X_test)
predicted_y.extend(predictions)
expected_y.extend(y_test)
report_test = classification_report(expected_y, predicted_y)
print('test set')
print(report_test)

In [ ]:
kfold = StratifiedKFold(n_splits=10, random_state=0)
results = cross_val_score(xgb_model_over, ls_sampling_build[0][1],ls_sampling_build[0][2], cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
 def build_xgbtune(self) :
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric="auc")
    predicted_y_train = []
    expected_y_train = []
    # Create grid search object
    clf = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, n_iter=200, cv=3, verbose=1, n_jobs=-1, return_train_score=True)
    # Fit on data
    best_clf = clf.fit(self.X_sam_train,self.Y_sam_train)
    self.best_hyparam = best_clf.best_params_
    logreg = xgb.XGBClassifier(**self.best_hyparam)
    logreg_build = logreg.fit(self.X_sam_train,self.Y_sam_train)
    predictions_train = logreg.predict(self.X_sam_train)
    predicted_y_train.extend(predictions_train)
    expected_y_train.extend(self.Y_sam_train)
    report_train_dic = classification_report(expected_y_train, predicted_y_train,output_dict=True)
    #Part test data
    predicted_y_test = []
    expected_y_test = []
    predictions_test = logreg.predict(self.X_sam_test)
    predicted_y_test.extend(predictions_test)
    expected_y_test.extend(self.Y_sam_test)
    report_test_dic = classification_report(expected_y_test, predicted_y_test,output_dict=True)
    return logreg_build,report_train_dic,report_test_dic ,"XGboost RandomSearch"

In [ ]:
%%time

def 
xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric="auc")
params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}


search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, n_iter=200, cv=3, verbose=1, n_jobs=-1, return_train_score=True)

search.fit(ls_sampling_build[0][1],ls_sampling_build[0][2])


In [ ]:
randint(2, 6)

In [ ]:
search.best_params_

In [ ]:
ssss_test = xgb.XGBClassifier(**search.best_params_)

In [ ]:
ls_sampling_build[0][3]

In [ ]:
ssss_test.fit(ls_sampling_build[0][1],ls_sampling_build[0][2], early_stopping_rounds=5, eval_set=[(ls_sampling_build[0][3].to_numpy(),ls_sampling_build[0][4])])
predicted_y = []
expected_y = []

# xgb_model_under = xgb_model.fit(X_undersampled, y_undersampled, early_stopping_rounds=5, eval_set=[(X_test.to_numpy(), y_test)])
predictions =  ssss_test.predict(ls_sampling_build[0][1])
predicted_y.extend(predictions)
expected_y.extend(ls_sampling_build[0][2])
report_train = classification_report(expected_y, predicted_y)
print('training set')
print(report_train) 

predicted_y = []
expected_y = []
predictions = ssss_test.predict(ls_sampling_build[0][3].to_numpy())
predicted_y.extend(predictions)
expected_y.extend(ls_sampling_build[0][4])
report_test = classification_report(expected_y, predicted_y)
print('test set')
print(report_test) 

In [ ]:
kfold = StratifiedKFold(n_splits=10, random_state=0)
results = cross_val_score(ssss_test, ls_sampling_build[0][3],ls_sampling_build[0][4], cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
report_best_scores(search.cv_results_, 1)

### **OVER**

### **DOWN**

### **SMOTE**

In [ ]:
#class model
class MachineLearning:

  def __init__(self,X_sam_train , Y_sam_train , X_sam_test , Y_sam_test) :
    self.X_sam_train = X_sam_train
    self.Y_sam_train = Y_sam_train
    self.X_sam_test = X_sam_test
    self.Y_sam_test = Y_sam_test
  
  def build_logistic(self,nd_tune=None) :
    if nd_tune != None :
      
    logreg = LogisticRegression(solver='liblinear', class_weight='balanced')
    #Part train data
    predicted_y_train = []
    expected_y_train = []
    logreg_build = logreg.fit(self.X_sam_train, self.Y_sam_train)
    predictions_train = logreg_build.predict(self.X_sam_train)
    predicted_y_train.extend(predictions_train)
    expected_y_train.extend(self.Y_sam_train)
    report_train_dic = classification_report(expected_y_train, predicted_y_train,output_dict=True)
    #Part test data
    predicted_y_test = []
    expected_y_test = []
    predictions_test = logreg_build.predict(self.X_sam_test)
    predicted_y_test.extend(predictions_test)
    expected_y_test.extend(self.Y_sam_test)
    report_test_dic = classification_report(expected_y_test, predicted_y_test)
    return logreg_build,report_train_dic,report_test_dic,"Logistic regression"

  def build_xgboost(self) :
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric="auc")
    predicted_y_train = []
    expected_y_train = []
    xgb_build = xgb_model.fit(self.X_sam_train, self.Y_sam_train, early_stopping_rounds=5, eval_set=[(self.X_sam_test.to_numpy(), self.Y_sam_test)])
    predictions_train = xgb_build.predict(self.X_sam_train)
    predicted_y_train.extend(predictions_train)
    expected_y_train.extend(self.Y_sam_train)
    report_train_dic = classification_report(expected_y_train, predicted_y_train,output_dict=True)
    #Part test data
    predicted_y_test = []
    expected_y_test = []
    predictions_test = xgb_build.predict(self.X_sam_test.to_numpy())
    predicted_y_test.extend(predictions_test)
    expected_y_test.extend(self.Y_sam_test)
    report_test_dic = classification_report(expected_y_test, predicted_y_test)
    return xgb_build,report_train_dic,report_test_dic,"XGboost"


In [ ]:
mas_model = MachineLearning()

## **LOGISTICS**

### **LOGISTICS - UnderSampling**

In [ ]:
print('logistic regression model - undersampled')
logreg = LogisticRegression(solver='liblinear', class_weight='balanced')

predicted_y = []
expected_y = []

logreg_model_under = logreg.fit(X_undersampled, y_undersampled)
predictions = logreg_model_under.predict(X_undersampled)
predicted_y.extend(predictions)
expected_y.extend(y_undersampled)
report_train = classification_report(expected_y, predicted_y)
print('training set')
print(report_train) 

predicted_y = []
expected_y = []
predictions = logreg_model_under.predict(X_test)
predicted_y.extend(predictions)
expected_y.extend(y_test)
report_test = classification_report(expected_y, predicted_y)
print('test set')
print(report_test) 

In [ ]:
y_score_train = logreg_model_under.decision_function(X_undersampled)
fpr_train, tpr_train, _ = roc_curve(y_undersampled,  y_score_train)
auc_train = roc_auc_score(y_undersampled, y_score_train)
plt.plot(fpr_train,tpr_train, color='red', label='undersampled - train , auc='+str(auc_train))

y_score_test = logreg_model_under.decision_function(X_test)
fpr_test, tpr_test, _ = roc_curve(y_test,  y_score_test)
auc_test = roc_auc_score(y_test, y_score_test)
plt.plot(fpr_test,tpr_test, color='Blue', label='undersampled - test , auc='+str(auc_test))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.legend(loc=4)
plt.show()

In [ ]:
print(logreg_model_under.coef_, logreg_model_under.intercept_)

### **LOGISTICS - OverSampling**

In [ ]:
print('logistic regression model - oversampled')
logreg = LogisticRegression(solver='liblinear', class_weight='balanced')
predicted_y = []
expected_y = []

logreg_model_over = logreg.fit(X_oversampled, y_oversampled)
predictions = logreg_model_over.predict(X_oversampled)
predicted_y.extend(predictions)
expected_y.extend(y_oversampled)
report_train = classification_report(expected_y, predicted_y)
print('training set')
print(report_train) 

predicted_y = []
expected_y = []
predictions = logreg_model_over.predict(X_test)
predicted_y.extend(predictions)
expected_y.extend(y_test)
report_test = classification_report(expected_y, predicted_y)
print('test set')
print(report_test) 

In [ ]:
y_score_train = logreg_model_over.decision_function(X_oversampled)
fpr_train, tpr_train, _ = roc_curve(y_oversampled,  y_score_train)
auc_train = roc_auc_score(y_oversampled, y_score_train)
plt.plot(fpr_train,tpr_train, color='red', label='oversampled - train , auc='+str(auc_train))

y_score_test = logreg_model_over.decision_function(X_test)
fpr_test, tpr_test, _ = roc_curve(y_test,  y_score_test)
auc_test = roc_auc_score(y_test, y_score_test)
plt.plot(fpr_test,tpr_test, color='Blue', label='oversampled - test , auc='+str(auc_test))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.legend(loc=4)
plt.show()

In [ ]:
print(logreg_model_over.coef_, logreg_model_under.intercept_)

### **LOGISTICS - SMOTE**

In [ ]:
print('logistic regression model - SMOTE')
logreg = LogisticRegression(solver='liblinear', class_weight='balanced')
predicted_y = []
expected_y = []

logreg_model_SMOTE = logreg.fit(X_SMOTE, y_SMOTE)
predictions = logreg_model_SMOTE.predict(X_SMOTE)
predicted_y.extend(predictions)
expected_y.extend(y_SMOTE)
report_train = classification_report(expected_y, predicted_y)
print('training set')
print(report_train) 

predicted_y = []
expected_y = []
predictions = logreg_model_SMOTE.predict(X_test)
predicted_y.extend(predictions)
expected_y.extend(y_test)
report_test = classification_report(expected_y, predicted_y)
print('test set')
print(report_test) 

In [ ]:
y_score_train = logreg_model_SMOTE.decision_function(X_SMOTE)
fpr_train, tpr_train, _ = roc_curve(y_SMOTE,  y_score_train)
auc_train = roc_auc_score(y_SMOTE, y_score_train)
plt.plot(fpr_train,tpr_train, color='red', label='SMOTE - train , auc='+str(auc_train))

y_score_test = logreg_model_SMOTE.decision_function(X_test)
fpr_test, tpr_test, _ = roc_curve(y_test,  y_score_test)
auc_test = roc_auc_score(y_test, y_score_test)
plt.plot(fpr_test,tpr_test, color='Blue', label='SMOTE - test , auc='+str(auc_test))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.legend(loc=4)
plt.show()

In [ ]:
print(logreg_model_SMOTE.coef_, logreg_model_under.intercept_)

## **XGBOOST**

### **XGBOOST-UnderSampling**

In [ ]:
print('XGBoost model - undersampled')
xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric="auc")
predicted_y = []
expected_y = []

xgb_model_under = xgb_model.fit(X_undersampled, y_undersampled, early_stopping_rounds=5, eval_set=[(X_test.to_numpy(), y_test)])
predictions =  xgb_model_under.predict(X_undersampled)
predicted_y.extend(predictions)
expected_y.extend(y_undersampled)
report_train = classification_report(expected_y, predicted_y)
print('training set')
print(report_train) 

predicted_y = []
expected_y = []
predictions = xgb_model_under.predict(X_test.to_numpy())
predicted_y.extend(predictions)
expected_y.extend(y_test)
report_test = classification_report(expected_y, predicted_y)
print('test set')
print(report_test) 

In [ ]:
y_score_train = xgb_model_under.predict_proba(X_undersampled)
fpr_train, tpr_train, _ = roc_curve(y_undersampled,  y_score_train[:,1])
auc_train = roc_auc_score(y_undersampled, y_score_train[:,1])
plt.plot(fpr_train,tpr_train, color='red', label='undersampled - train , auc='+str(auc_train))

y_score_test = xgb_model_under.predict_proba(X_test.to_numpy())
fpr_test, tpr_test, _ = roc_curve(y_test,  y_score_test[:,1])
auc_test = roc_auc_score(y_test, y_score_test[:,1])
plt.plot(fpr_test,tpr_test, color='Blue', label='undersampled - test , auc='+str(auc_test))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.legend(loc=4)
plt.show()

In [ ]:
xgb.plot_importance(xgb_model_under)

### **XGBOOST-OverSampling**

In [ ]:
report_best_scores(search.cv_results_, 1)

In [ ]:
search.best_params_

In [ ]:
print('XGBoost model - oversampled')
xgb_model = xgb.XGBClassifier(**search.best_params_)
predicted_y = []
expected_y = []

xgb_model_over = xgb_model.fit(oversam_build[1], oversam_build[2], early_stopping_rounds=5, eval_set=[(X_test.to_numpy(), y_test)])
predictions =  xgb_model_over.predict(oversam_build[1])
predicted_y.extend(predictions)
expected_y.extend(oversam_build[2])
report_train = classification_report(expected_y, predicted_y)
print('training set')
print(report_train) 

predicted_y = []
expected_y = []
predictions = xgb_model_over.predict(X_test.to_numpy())
predicted_y.extend(predictions)
expected_y.extend(y_test)
report_test = classification_report(expected_y, predicted_y)
print('test set')
print(report_test) 

In [ ]:
y_score_train = xgb_model_over.predict_proba(oversam_build[1])
fpr_train, tpr_train, _ = roc_curve(oversam_build[2],  y_score_train[:,1])
auc_train = roc_auc_score(oversam_build[2], y_score_train[:,1])
plt.plot(fpr_train,tpr_train, color='red', label='oversampled - train , auc='+str(auc_train))

y_score_test = xgb_model_over.predict_proba(X_test.to_numpy())
fpr_test, tpr_test, _ = roc_curve(y_test,  y_score_test[:,1])
auc_test = roc_auc_score(y_test, y_score_test[:,1])
plt.plot(fpr_test,tpr_test, color='Blue', label='oversampled - test , auc='+str(auc_test))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.legend(loc=4)
plt.show()

In [ ]:
xgb.plot_importance(xgb_model_over)

### **XGBOOST-SMOTE**

In [ ]:
print('XGBoost model - SMOTE')

xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric="auc")
predicted_y = []
expected_y = []

xgb_model_SMOTE = xgb_model.fit(X_SMOTE, y_SMOTE, early_stopping_rounds=5, eval_set=[(X_test.to_numpy(), y_test)])
predictions =  xgb_model_SMOTE.predict(X_SMOTE)
predicted_y.extend(predictions)
expected_y.extend(y_SMOTE)
report_train = classification_report(expected_y, predicted_y)
print('training set')
print(report_train) 

predicted_y = []
expected_y = []
predictions = xgb_model_SMOTE.predict(X_test.to_numpy())
predicted_y.extend(predictions)
expected_y.extend(y_test)
report_test = classification_report(expected_y, predicted_y)
print('test set')
print(report_test) 

In [ ]:
y_score_train = xgb_model_SMOTE.predict_proba(X_SMOTE)
fpr_train, tpr_train, _ = roc_curve(y_SMOTE,  y_score_train[:,1])
auc_train = roc_auc_score(y_SMOTE, y_score_train[:,1])
plt.plot(fpr_train,tpr_train, color='red', label='SMOTE - train , auc='+str(auc_train))

y_score_test = xgb_model_SMOTE.predict_proba(X_test.to_numpy())
fpr_test, tpr_test, _ = roc_curve(y_test,  y_score_test[:,1])
auc_test = roc_auc_score(y_test, y_score_test[:,1])
plt.plot(fpr_test,tpr_test, color='Blue', label='SMOTE - test , auc='+str(auc_test))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.legend(loc=4)
plt.show()

In [ ]:
xgb.plot_importance(xgb_model_SMOTE)

In [ ]:
print('XGBoost model - SMOTE - parameter tuning')

xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric="auc",
 base_score=0.6,
 learning_rate =0.02,
 n_estimators=8000,
 max_depth=3,
 min_child_weight=2,
 gamma=0.1,
 subsample=0.4,
 colsample_bytree=0.4,
 nthread=4)
predicted_y = []
expected_y = []

xgb_model_SMOTE_tuned = xgb_model.fit(X_SMOTE, y_SMOTE, early_stopping_rounds=100, eval_set=[(X_test.to_numpy(), y_test)])
predictions =  xgb_model_SMOTE_tuned.predict(X_SMOTE)
predicted_y.extend(predictions)
expected_y.extend(y_SMOTE)
report_train = classification_report(expected_y, predicted_y)
print('training set')
print(report_train) 

predicted_y = []
expected_y = []
predictions = xgb_model_SMOTE_tuned.predict(X_test.to_numpy())
predicted_y.extend(predictions)
expected_y.extend(y_test)
report_test = classification_report(expected_y, predicted_y)
print('test set')
print(report_test) 

In [ ]:
tn, fp, fn, tp = confusion_matrix(expected_y, predicted_y).ravel()
df_cfm = pd.DataFrame({"True":[tp,tn],"False":[fp,fn]})
df_cfm.index = ["Model -> Positive" , "Model -> Negative"]
df_cfm

In [ ]:
y_score_train = xgb_model_SMOTE_tuned.predict_proba(X_SMOTE)
fpr_train, tpr_train, _ = roc_curve(y_SMOTE,  y_score_train[:,1])
auc_train = roc_auc_score(y_SMOTE, y_score_train[:,1])
plt.plot(fpr_train,tpr_train, color='red', label='SMOTE - train , auc='+str(auc_train))

y_score_test = xgb_model_SMOTE.predict_proba(X_test.to_numpy())
fpr_test, tpr_test, _ = roc_curve(y_test,  y_score_test[:,1])
auc_test = roc_auc_score(y_test, y_score_test[:,1])
plt.plot(fpr_test,tpr_test, color='Blue', label='SMOTE - test , auc='+str(auc_test))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.legend(loc=4)
plt.show()